## EDA

In [ ]:
#load python lobraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom # handle dcm format
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom # handle dcm format
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

In [ ]:
# !mkdir output
# df = pd.read_csv('../input/submission-file/sample_submission (2).csv')
# df.to_csv('output/submission.csv',index=False)
# df.to_csv('submission.csv',index=False)
# pd.read_csv('output/submission.csv')

In [ ]:
#material (dicom mri images deep learning , dicom image processing python)
#https://medium.com/analytics-vidhya/dicom-and-deep-learning-63373e99d79a
#https://towardsdatascience.com/understanding-dicoms-835cd2e57d0b
#important features we can extract from dicom fileset
#dicom mri images deep learning , dicom image processing python ,important header elements in diacom files for deep learning
#important features we can extract from dicom files
#important header elements in diacom files for deep learning
#https://www.researchgate.net/post/Convolutional_Neural_network_for_stack_of_CT_or_MRI_images/61812b52b8ca5f507e065c3b/citation/download
#https://towardsdatascience.com/understanding-dicom-bce665e62b72
#https://towardsdatascience.com/medical-image-pre-processing-with-python-d07694852606

### Load the data into dataframe

In [ ]:
#load the training lables
train_label_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
train_label_df

## check class imbalance

In [ ]:
#check class imbalance
plt.figure(figsize=(5, 5))
sns.countplot(data=train_label_df, x="MGMT_value");

- data set is balance dataset

### Headers in dicom images

In [ ]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    return dicom
load_dicom('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-1.dcm')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom # handle dcm format
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

### create dataframe with headers for eda

In [ ]:
train_label_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')

imp_headers = ['ImageType','Modality','SliceThickness','PatientPosition','StudyInstanceUID','SeriesInstanceUID','PhotometricInterpretation' ,'Rows','Columns','PixelSpacing','RescaleSlope','RescaleIntercept']

types = ("FLAIR", "T1w", "T1wCE", "T2w")
column_list = ['BraTS21ID']
for file_type in types:
    for header in imp_headers:
        column_list.append(file_type+'_'+header)
column_list.append('MGMT_value')
header_df = pd.DataFrame(columns=column_list)
column_list
def fill_header_df(datapoints):
    for point_ind , datapoint in enumerate(datapoints):
        count = point_ind
        df_row = []
        folder_id = str(datapoint).zfill(5)
        df_row.append(folder_id)
        for type_index, type_val in enumerate(types, 1):
            
            patient_path = f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{folder_id}/'
            all_paths = sorted(
                #glob used to return all file paths that match a specific pattern.
                glob.glob(os.path.join(patient_path, type_val, "*")), 
                key=lambda x: int(x[:-4].split("-")[-1]),
            )
            img_path = all_paths[int(len(all_paths) * 0.5)]
            dicom_img = pydicom.read_file(img_path)
            for header in imp_headers:
                df_row.append(dicom_img[header].value)
        df_row.append(train_label_df.loc[point_ind,'MGMT_value'])
        header_df.loc[count] = df_row
fill_header_df(train_label_df['BraTS21ID'])
os.mkdir('output')
header_df.to_csv('output/header_df.csv',index=False)
header_df.head()


In [ ]:
header_df = pd.read_csv('output/header_df.csv', index_col= False)

In [ ]:
header_df.head()

## check null values

In [ ]:
header_df.isnull().sum()

there are no null values in any column

## plot the hisogram to count the values for  SliceThickness

In [ ]:
#pixel spacing and slice thickness
#plot hsitogram and eda

#each feature with no of unique values
# two features histograms and distribution 
types = ('FLAIR','T1w' , 'T1wCE' , 'T2w')
for image_type in types:
    FLAIR_SliceThickness = header_df.FLAIR_SliceThickness.value_counts()
    #plt.hist(header_df['FLAIR_SliceThickness'])
    
    sns.histplot(data=header_df, x=f"{image_type}_SliceThickness", hue="MGMT_value", multiple="stack")
    plt.xlabel(f"{image_type}_SliceThickness")
    plt.ylabel('counts')
    plt.show()
    header_df.FLAIR_SliceThickness.describe()

## Plot distribution graph for slice thick ness

In [ ]:
#loop through
for image_type in types:
    sns.distplot(header_df[header_df['MGMT_value'] == 0][f'{image_type}_SliceThickness'], hist=False, color="blue" )
    sns.distplot(header_df[header_df['MGMT_value'] == 1][f'{image_type}_SliceThickness'], hist=False, color="orange")
    plt.legend([0,1]) 
    plt.title(f'distribution of {image_type}_SliceThickness') 
    plt.show()
    header_df[f'{image_type}_SliceThickness'].describe()

## check stats for pixel spacing 

In [ ]:
# check stats for pixel spacing 
for image_type in types:
    print(header_df[f'{image_type}_PixelSpacing'].describe(),'\n'*2)


## Median and mode for pixel spacing  and slice thickness

In [ ]:
for image_type in types:
    
    stats = header_df[f'{image_type}_PixelSpacing'].apply(lambda x:eval(x)[0] ).describe()
    stats['median'] = header_df[f'{image_type}_PixelSpacing'].apply(lambda x:eval(x)[0] ).median()
    print(stats)

In [ ]:
for image_type in types:
    
    stats = header_df[f'{image_type}_SliceThickness'].describe()
    stats['median'] = header_df[f'{image_type}_SliceThickness'].median()
    print(stats)

In [ ]:
#pairrwise histogram
for image_type in types:
    value_counts = header_df[f'{image_type}_PixelSpacing'].value_counts()
    # value_counts
    top_header_df = header_df[header_df[f'{image_type}_PixelSpacing'].isin(value_counts.keys())]
    PixelSpacing = top_header_df[f'{image_type}_PixelSpacing'].apply(lambda x:round(eval(x)[0],3) )
    plt.figure(figsize=(16,8))
    sns.histplot(data=top_header_df, x=PixelSpacing, hue="MGMT_value", multiple="dodge" )

### we need this plots and stats of pixel spacing and slice thickness to choose pixel spacing and sclice thickness to resample images it will give better model performance

## keep Columns with more than 1 unique values only

In [ ]:
value_counts = header_df['FLAIR_PixelSpacing'].value_counts()
print(value_counts)

In [ ]:
# loop through and exclude unique values , discrd those features who have only one unique values
cloumns_exclude = [(column , len(header_df[column].value_counts()) ) for column in header_df.columns if len(header_df[column].value_counts()) > 1]
cloumns_exclude

In [ ]:
columns_to_keep = [column for column in header_df.columns if len(header_df[column].value_counts()) > 1]
final_header_df = header_df.loc[:,header_df.columns.isin(columns_to_keep)]
final_header_df.head()

## Model building


In [ ]:
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom # handle dcm format
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
#from models import resnet

## sklearn
from sklearn import model_selection
from sklearn.metrics import roc_auc_score

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, X_train_data , y_train_data , data_dir,  cohort='FLAIR'):
        self.X_train_data = X_train_data
        self.y_train_data = y_train_data
        self.data_dir = data_dir
        self.cohort = cohort
    def __getitem__(self, idx):
        # get sample info
        #print(self.X_train_data[idx],'idx')
        sample_id = self.X_train_data[idx] 
        target = self.y_train_data[idx]
        # get sample path. combination of dir, padded id and cohort
        sample_dir = os.path.join(self.data_dir, f'{sample_id:05d}', self.cohort)
        sample_files = os.listdir(sample_dir)
        #print(sample_dir,'sample_files')
        # take subset of available images if n_images > 64
        if len(sample_files) > 64:
            sample_files = np.random.choice(sample_files, size=64, replace=False)
            
        # sort samples
        sample_files = sorted(sample_files, key=lambda x: int(x[6:-4]))
        
        # load images
        imgs = [self.read_img(os.path.join(sample_dir, path)) for path in sample_files]
        imgs = np.stack(imgs)
        
        # resample images if not enough samples are available
        if len(sample_files) < 64:
            indices = sorted(np.random.choice(len(sample_files), size=64, replace=True))
            imgs = np.stack(imgs[indices])
        
        imgs = np.stack(imgs)
            
        return torch.tensor(imgs, dtype=torch.float32).unsqueeze(0), torch.tensor(target, dtype=torch.float32)
        
    def __len__(self):
        return len(self.X_train_data)
    
    def read_img(self, path):
        img = self.read_dicom(path)
        img = cv2.resize(img, (258, 258))
        return img
    
    @staticmethod
    def read_dicom(path):
        dicom = pydicom.read_file(path)
        data = apply_voi_lut(dicom.pixel_array, dicom)
        if dicom.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
        data = data - np.min(data)
        data = data / np.max(data)
        data = (data * 255).astype(np.uint8)
        return data

In [ ]:
#nifti dataloader
import numpy as np
import nibabel as nib
class NiftiDataset(Dataset):
    def __init__(self, X_train_data , y_train_data, data_dir,  cohort='FLAIR'):
        self.X_train_data = X_train_data
        self.y_train_data = y_train_data
        self.data_dir = data_dir
        self.cohort = cohort
    def __getitem__(self, idx):
        # get sample info
        sample_id = self.X_train_data[idx] 
        
        if type(self.y_train_data) == type(None):
            target = []
        else:
            target = self.y_train_data[idx]
        # get sample path. combination of dir, padded id and cohort
        sample_dir = os.path.join(self.data_dir, f'{sample_id:05d}', self.cohort)
        #final_tensor = get_padded_data(sample_dir , f'{sample_id:05d}')
        img = nib.load(f'{sample_dir}/{sample_id:05d}_FLAIR.nii.gz')
        final_tensor = img.get_fdata()
        final_tensor = final_tensor - np.min(final_tensor)
        final_tensor = final_tensor / np.max(final_tensor)
        return torch.tensor(final_tensor, dtype=torch.float32).unsqueeze(0), torch.tensor(target, dtype=torch.float32)
        
    def __len__(self):
        return len(self.X_train_data)
    

In [ ]:
#load the training lables
from sklearn.model_selection import train_test_split
train_label_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
train_label_df = train_label_df[train_label_df.BraTS21ID.isin(list(map(int, os.listdir('../input/dicom-to-normalized-nifti-with-torchio/processed/train/') )))]

X = train_label_df['BraTS21ID']
y = train_label_df['MGMT_value']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, stratify=y)
X_test = X_test.reset_index(drop= True)
y_test = y_test.reset_index(drop= True)
X_train = X_train.reset_index(drop= True)
y_train = y_train.reset_index(drop= True)

In [ ]:
#!rm -rf MedicalNet
#!git clone https://github.com/Tencent/MedicalNet

In [ ]:
#pip install -r ./MedicalNet/requirements.txt

In [ ]:
import sys
import os
import random
import time

#run following steps
os.system('cp -r ../input/medical-net-files/data/ ../input/medical-net/MedicalNet-master')
os.system('cp -r ../input/medical-net-files/pretrain/ ../input/medical-net/MedicalNet-master')
os.system('cp -r ../input/medical-net-files/trails/ ../input/medical-net/MedicalNet-master')
os.system('touch ../input/medical-net/MedicalNet-master/__init__.py')
sys.path.append('../input/medical-net/MedicalNet-master')
import MedicalNet.model

In [ ]:
class CFG:
    # system
    seed=42
    no_cuda=False
    
    # model
    model_name = 'resnet_34_23dataset'
    use_pretrained = True
    
class TrainerConfig:
    num_epochs = 20
    batch_size = 4
    gradient_accumulation_steps = 1

    # optimizer
    lr = 3e-2
    warm_up_ratio = 0.1
    weight_decay = 0.0

    # log every log_steps to wandb
    log_steps = 20

    # environment
    device = 'cuda'
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
# MedicalNet expect a class to hold hyperparameters
class Struct:
    def __init__(self, entries):
        self.__dict__.update(entries)

In [ ]:
# system setup
seed_everything(CFG.seed)
DEVICE = torch.device("cuda") if torch.cuda.is_available() and not CFG.no_cuda else torch.device("cuda")
DEVICE = torch.device("cuda")

In [ ]:
# model specific args
model_pretrained_params = {
    'resnet_10': {'model_depth': 10, 'resnet_shortcut': 'B'},
    'resnet_10_23dataset': {'model_depth': 10, 'resnet_shortcut': 'B'},
    'resnet_18': {'model_depth': 18, 'resnet_shortcut': 'A'},
    'resnet_18_23dataset': {'model_depth': 18, 'resnet_shortcut': 'A'},
    'resnet_34': {'model_depth': 34, 'resnet_shortcut': 'A'},
    'resnet_34_23dataset': {'model_depth': 34, 'resnet_shortcut': 'A'},
    'resnet_50': {'model_depth': 50, 'resnet_shortcut': 'B'},
    'resnet_50_23dataset': {'model_depth': 50, 'resnet_shortcut': 'B'},
    'resnet_101': {'model_depth': 101, 'resnet_shortcut': 'B'},
    'resnet_152': {'model_depth': 152, 'resnet_shortcut': 'B'},
    'resnet_200': {'model_depth': 200, 'resnet_shortcut': 'B'},
}
# consistent args
opts = {
    'model': 'resnet',
    'input_W': 256,
    'input_H': 256,
    'input_D': 64,
    'no_cuda': CFG.no_cuda,
    'n_seg_classes': 1,
    'phase': 'train',
    'pretrain_path': None,
    'gpu_id': [1],
}

# merge modelspecific args and global args
for model_name, model_dict in model_pretrained_params.items():
    model_pretrained_params[model_name] = Struct({**model_dict, **opts})
    
    
# MedicalNet with a global pooling head
class MedicalNetWithHead(nn.Module):
    def __init__(self, model_name, pretrain_path=None):
        super().__init__()
        self.model_name = model_name
        
        model, parameters = MedicalNet.model.generate_model(model_pretrained_params[model_name])
        self.medical_net = model
        self.drop_in = nn.Dropout(p=0.1)
        
        # init model with pretrained weights
        if not pretrain_path and CFG.use_pretrained:
            self.init_model()
            
        # use simple pooling for now
        self.pool = nn.AdaptiveAvgPool3d(1)
        
    def forward(self, x):
        x = self.medical_net(self.drop_in(x))
        out = self.pool(x)
        return out
            
    def init_model(self):
        net_dict = self.medical_net.state_dict()
        # load pretrain
        pretrain = torch.load(f'../input/medical-net/MedicalNet-master/pretrain/{self.model_name}.pth', map_location=DEVICE)
        net_dict.update(pretrain_dict)
        self.medical_net.load_state_dict(net_dict)
        print("loaded pretrained weights")

In [ ]:
def collate_fn(data):
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    height_max , width_max , depth_max  = 0,0,0
    data_return = []
    lebels_return = []
    for ind , data_point in enumerate(data):
        ind , height , width , depth  = data[ind][0].shape
        height_max = max(height_max ,height )
        width_max = max(width_max ,width )
        depth_max = max(depth_max ,depth )
    for ind,data_tensor in enumerate(data):
        padded_data = torch.zeros((1,height_max,width_max ,depth_max))
        padded_data[0,:data[ind][0].shape[1],:data[ind][0].shape[2],:data[ind][0].shape[3]] = data_tensor[0][0]
        
        data_return.append(padded_data)
        if data[ind][1].nelement():
            lebels_return.append(data[ind][1])
    return torch.cat(data_return ,0).unsqueeze(1), torch.tensor(lebels_return)

In [ ]:
!pip install tensorboard
#!tensorboard --logdir=runs

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
class AverageMeter(object):
    """
    Computes and stores the average and current value
    Copied from: https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class Trainer:
    def __init__(self, cfg: type(TrainerConfig),
                 model: torch.nn.Module,
                 model_path: str,
                 dataset_train: Dataset = None,
                 dataset_val: Dataset = None,
                 #wandb_run: wandb.sdk.wandb_run.Run = None
                 ):
        self.cfg = cfg
        self.model = model
        self.best_model = None
        self.model_path = model_path
        #self.wandb_run = wandb_run

        # datasets
        self.dataset_train = dataset_train
        self.dataset_eval = dataset_val

        # dataloaders, train/eval is optional
        kwargs_dataloader = {'batch_size': self.cfg.batch_size, 'num_workers': 2 , 'collate_fn' : collate_fn}
        if self.dataset_eval is not None:
            self.dataloader_train = DataLoader(self.dataset_train, shuffle=True, **kwargs_dataloader  )
        if self.dataset_eval is not None:
            self.dataloader_eval = DataLoader(self.dataset_eval, shuffle=False, **kwargs_dataloader)

        # setup loss
        self.loss_fnc = torch.nn.BCEWithLogitsLoss()

        # if train set is provided, setup training
        if dataset_train is not None:
            self.optimizer = torch.optim.SGD(self.model.parameters(), self.cfg.lr, weight_decay=self.cfg.weight_decay, momentum=0.9)
            #self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.0001)
            self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.9)
            # call optimizer once so we can properly init the lr in step_train()
            self.optimizer.zero_grad()
            self.optimizer.step()
            
        else:
            self.optimizer = None
            self.scheduler = None
            
        self.epoch = 0

    def train(self):
        print(f'Training model for {self.cfg.num_epochs} epochs.')
        print('Epoch | train_loss | eval_loss | train_auc | val_auc')
        train_log = pd.DataFrame(columns=['epoch', 'train_loss', 'eval_loss', 'train_auc', 'eval_auc', 'lr'])
        
        best_loss = 1e3

        for epoch in range(self.cfg.num_epochs):
            train_loss, train_auc = self.step_train()
            eval_loss, eval_auc = self.step_eval(return_predictions=False)
            writer.add_scalar('Loss_epoch/train', train_loss, self.epoch)
            writer.add_scalar('auc_epoch/train', train_auc,self.epoch)
            writer.add_scalar('Loss/test', eval_loss, self.epoch)
            writer.add_scalar('auc/test', eval_auc, self.epoch)
            
            log_item = {
                'epoch': epoch,
                'step': epoch*len(self.dataloader_train),
                'train_loss': train_loss,
                'eval_loss': eval_loss,
                'train_auc': train_auc,
                'eval_auc': eval_auc,
                'lr': self.optimizer.param_groups[0]['lr']
            }
            #self.wandb_run.log(log_item)
            train_log = train_log.append(log_item, ignore_index=True)
            print(f"{epoch: <6}|{train_loss: >12.3f}|{eval_loss: >11.3f}|{train_auc: >11.3f}|{eval_auc: >8.3f}")
      
            # checkpointing
            if eval_loss < best_loss:
                torch.save(self.model, self.model_path)
                best_loss = eval_loss
            self.epoch += 1
        best_epoch = train_log.eval_loss.idxmin()
        print("Training done. Best model at epoch {} with eval_loss {:3.2f} and auc {:3.2f}".format(
            train_log.loc[best_epoch, 'epoch'],
            train_log.loc[best_epoch, 'eval_loss'],
            train_log.loc[best_epoch, 'eval_auc']
        ))
        return train_log

    def step_train(self):
        self.model.train()
        
        # setup logging
        loss_agg = AverageMeter()
        targets = []
        predictions = []
        steps_per_ep = len(self.dataset_train)//self.cfg.batch_size +1
        # train one epoch
        for batch_idx, (x, y) in enumerate(self.dataloader_train):
            
            x = x.to(self.cfg.device)
            y = y.to(self.cfg.device)
            
            # forward pass  
            logits = self.model(x)
            loss = self.loss_fnc(logits.view(-1), y)
            writer.add_scalar('Loss_step/train', loss, self.epoch*steps_per_ep + batch_idx)
            batch_pred = torch.sigmoid(logits.view(-1)).detach().cpu().squeeze().numpy()
            batch_tar  = y.detach().cpu().squeeze().numpy()
            #batch_auc = roc_auc_score(batch_tar, batch_pred)
            #writer.add_scalar('auc_step/train', batch_auc, self.epoch*steps_per_ep + batch_idx)
            # backward pass
            loss.backward()
            if (batch_idx+1) % self.cfg.gradient_accumulation_steps == 0 or (batch_idx+1) == len(self.dataloader_train):
                self.optimizer.step()
                self.optimizer.zero_grad()
                self.scheduler.step()

            # update loss meter
            loss_agg.update(loss.item(), self.cfg.batch_size)

            # save preds/targets for roc computation
            predictions.append(batch_pred)
            targets.append(batch_tar)

            # log every cfg.log_steps steps
            if batch_idx > 0 and batch_idx % self.cfg.log_steps == 0:
                log_item = {
                    'step': self.epoch*len(self.dataloader_train) + batch_idx,
                    'train_loss': loss_agg.avg,
                    'train_auc': roc_auc_score(np.hstack(targets), np.hstack(predictions)),
                    'lr': self.optimizer.param_groups[0]['lr']
                }
                #self.wandb_run.log(log_item)
            
        # compute auc for whole epoch
        auc = roc_auc_score(np.hstack(targets), np.hstack(predictions))
        
        # tesorboard 
        
        torch.cuda.empty_cache()
        return loss_agg.avg, auc

    @torch.no_grad()
    def step_eval(self, return_predictions=False):
        self.model.eval()
        loss_agg = AverageMeter()
        predictions = []
        targets = []
        for batch_idx, (x, y) in enumerate(self.dataloader_eval):
            x = x.to(self.cfg.device)
            y = y.to(self.cfg.device)
            logits = self.model(x)
            loss = self.loss_fnc(logits.view(-1), y)

            # update loss meter
            loss_agg.update(loss.item(), self.cfg.batch_size)

            # optionally return predictions
            # save preds/targets for roc computation
            predictions.append(torch.sigmoid(logits.view(-1)).detach().cpu().squeeze().numpy())
            targets.append(y.detach().cpu().squeeze().numpy())
            
        # compute auc
        targets = np.hstack(targets)
        predictions = np.hstack(predictions)
        auc = roc_auc_score(targets, predictions)
        # setup output
        if return_predictions:
            out = (loss_agg.avg, auc, predictions)
        else:
            out = (loss_agg.avg, auc)
        return out
    
    def predict_results(self):
        self.model.eval()
        loss_agg = AverageMeter()
        predictions = []
        targets = []
        for batch_idx, (x, y) in enumerate(self.dataloader_eval):
            x = x.to(self.cfg.device)
            logits = self.model(x)
            # optionally return predictions
            # save preds/targets for roc computation
            predictions.append(torch.sigmoid(logits.view(-1)).detach().cpu().squeeze().numpy())

        # get predictions
        predictions = np.hstack(predictions)

        return predictions
    

In [ ]:
#model training
training_data = NiftiDataset(X_train,y_train, data_dir='../input/dicom-to-normalized-nifti-with-torchio/processed/train', cohort='FLAIR')
test_data = NiftiDataset(X_test,y_test, data_dir='../input/dicom-to-normalized-nifti-with-torchio/processed/train', cohort='FLAIR')

#DEVICE = torch.device("cuda")
model = MedicalNetWithHead(model_name=CFG.model_name).to(DEVICE)

# setup trainer
trainer = Trainer(cfg=TrainerConfig,
                  model=model,
                  model_path=f'model_save.torch',
                  dataset_train=training_data,
                  dataset_val=test_data,
                  #wandb_run=wandb_run
                 )
train_log = trainer.train()
#save the model
train_log.to_csv(f"model_log.csv")
torch.save(model.state_dict(), './MedicalNet/final_model' )
# added inference funtion


## inference funtion

In [ ]:
path_to_test_data = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test'
def create_submission_csv(path_to_test_data):
    test_eval_data =  list(map(int,list(os.listdir(path_to_test_data))))
    test_eval_dataset = NiftiDataset(test_eval_data, None, data_dir='../input/dicom-to-normalized-nifti-with-torchio/processed/test', cohort='FLAIR')
    #test_eval_dataloader = DataLoader(test_eval_dataset, batch_size=3, shuffle=True)
    model_test_pred= MedicalNetWithHead(model_name=CFG.model_name).to(DEVICE)
    model_test_pred.load_state_dict(torch.load('../input/final-modal/model_submit.torch'))
    #model_test_pred.eval()
    trainer_pred = Trainer(cfg=TrainerConfig,
                      model=model_test_pred,
                      model_path=f'model_save.torch',
                      dataset_train=training_data,
                      dataset_val=test_eval_dataset,
                      #wandb_run=wandb_run
                     )
    test_data_pred = trainer_pred.predict_results()
    df = pd.DataFrame({'BraTS21ID': list(map(lambda sample_id: (5 - len(str(sample_id))) * '0'+str(sample_id) ,test_eval_data)),
                       'MGMT_value': test_data_pred , 
                      })
    df=df.set_index('BraTS21ID')
    df.to_csv('submission.csv')
print(test_data_pred)

In [ ]:
create_submission_csv(path_to_test_data)